In [47]:
import os
import tkinter as tk
import pandas as pd
import geopandas as gpd
import folium
import matplotlib.pyplot as plt
from shapely.geometry import mapping
from tkinter import filedialog
from shapely.geometry import Point, Polygon, LineString
import webbrowser
from shapely.geometry import MultiPolygon
import earthaccess
from shapely.geometry.polygon import orient
from shapely.geometry.polygon import orient


In [48]:
# load the csv file use a dialog box (from TKinter user guuide)
root = tk.Tk()
root.withdraw()
file = filedialog.askopenfilename()
df = pd.read_csv(file, encoding='latin1')

In [49]:
# load the csv into a dataframe df
# Show the first 10 rows of the csv dataset  - to let me understand what is in it
print("First 10 rows of the data:")
print(df.head(10))

print("\n-----------------------------\n")

# Count how many rows are in the data, this will let you know what happens when it is cleaned
row_count = len(df)
print("Total number of rows:", row_count)

# Count how many columns are in the data so that you can understand the size and shape of the csv dataset for the terrorism
col_count = len(df.columns)
print("Total number of columns:", col_count)

First 10 rows of the data:
     ï»¿eventid  iyear  imonth  iday approxdate  extended  resolution  \
0  1.970010e+11   1970       1     0        NaN         0         NaN   
1  1.970030e+11   1970       3    31        NaN         1  03/04/1970   
2  1.971110e+11   1971      11    20        NaN         0         NaN   
3  1.973040e+11   1973       4    25        NaN         0         NaN   
4  1.973080e+11   1973       8    29        NaN         0         NaN   
5  1.974070e+11   1974       7    15        NaN         0         NaN   
6  1.974080e+11   1974       8    15        NaN         0         NaN   
7  1.974110e+11   1974      11    14        NaN         0         NaN   
8  1.974110e+11   1974      11    22        NaN         0         NaN   
9  1.975020e+11   1975       2    28        NaN         0         NaN   

   country  country_txt  region region_txt provstate      city   latitude  \
0      101        Japan       4  East Asia   Fukouka   Fukouka  33.580412   
1      101     

In [50]:
# to reduce the date three colums into one to limit the impact processing needed  (Big o ) notation) 
if 'iyear' in df and 'imonth' in df and 'iday' in df:
    df['date'] = pd.to_datetime(
        dict(year=df['iyear'], month=df['imonth'], day=df['iday']), errors='coerce')

In [51]:
# Show the first 10 rows to check the date has been combined in the data frame
print(df.head(10))

     ï»¿eventid  iyear  imonth  iday approxdate  extended  resolution  \
0  1.970010e+11   1970       1     0        NaN         0         NaN   
1  1.970030e+11   1970       3    31        NaN         1  03/04/1970   
2  1.971110e+11   1971      11    20        NaN         0         NaN   
3  1.973040e+11   1973       4    25        NaN         0         NaN   
4  1.973080e+11   1973       8    29        NaN         0         NaN   
5  1.974070e+11   1974       7    15        NaN         0         NaN   
6  1.974080e+11   1974       8    15        NaN         0         NaN   
7  1.974110e+11   1974      11    14        NaN         0         NaN   
8  1.974110e+11   1974      11    22        NaN         0         NaN   
9  1.975020e+11   1975       2    28        NaN         0         NaN   

   country  country_txt  region region_txt provstate      city   latitude  \
0      101        Japan       4  East Asia   Fukouka   Fukouka  33.580412   
1      101        Japan       4  East Asia

In [52]:
# Count the total number of rows to show how many incidents there are
print("\nTotal number of rows:", len(df))


Total number of rows: 847


In [53]:
# Count how many columns are in the csv dataset
col_count = len(df.columns)
print("Total number of columns:", col_count)

Total number of columns: 20


In [54]:
# Keep only the columns that are needed for the project
columns_to_keep = ['region_txt', 'country_txt','city',  'date', 'latitude', 'longitude', 'summary']
df = df[columns_to_keep]



In [55]:
# Count how many columns are in the csv dataset
col_count = len(df.columns)
print("Total number of columns:", col_count)

# Show the first 10 rows to check the columns have been removed from the data frame
print(df.head(10))

Total number of columns: 7
  region_txt  country_txt      city       date   latitude   longitude summary
0  East Asia        Japan   Fukouka        NaT  33.580412  130.396361     NaN
1  East Asia        Japan   Fukouka 1970-03-31  33.580412  130.396361     NaN
2  East Asia       Taiwan   Unknown 1971-11-20  23.583333  119.583330     NaN
3  East Asia        Japan  Yokosuka 1973-04-25  35.281341  139.672200     NaN
4  East Asia        Japan     Tokyo 1973-08-29  35.689125  139.747742     NaN
5  East Asia        Japan     Itami 1974-07-15  34.784306  135.400947     NaN
6  East Asia  South Korea     Seoul 1974-08-15  37.566535  126.977969     NaN
7  East Asia        Japan     Tokyo 1974-11-14  35.689125  139.747742     NaN
8  East Asia        Japan   Fukouka 1974-11-22  33.580412  130.396361     NaN
9  East Asia        Japan     Tokyo 1975-02-28  35.689125  139.747742     NaN


In [56]:
# Remove rows with missing latitude or longitude - as points need to be plotted
if 'latitude' in df and 'longitude' in df:
    df = df.dropna(subset=['latitude', 'longitude'])

In [57]:
# Count the total number of rows to show how many incidents there are
print("\nTotal number of rows:", len(df))


Total number of rows: 839


In [58]:
# lowercase text for all the columns to make handelling text easier if needed
for col in df.columns:
    if df[col].dtype == object:
        df[col] = df[col].str.lower().str.strip()

In [59]:
# drop duplicates
df.drop_duplicates(inplace=True)

# Count the total number of rows do see how many duplicate records there were
print("\nTotal number of rows:", len(df))


Total number of rows: 694


In [60]:
# Show a few rows to check if text is lowercased
print(df.head(5))  # Shows first 5 rows

  region_txt country_txt      city       date   latitude   longitude summary
0  east asia       japan   fukouka        NaT  33.580412  130.396361     NaN
1  east asia       japan   fukouka 1970-03-31  33.580412  130.396361     NaN
2  east asia      taiwan   unknown 1971-11-20  23.583333  119.583330     NaN
3  east asia       japan  yokosuka 1973-04-25  35.281341  139.672200     NaN
4  east asia       japan     tokyo 1973-08-29  35.689125  139.747742     NaN


In [61]:
print(df.head())


  region_txt country_txt      city       date   latitude   longitude summary
0  east asia       japan   fukouka        NaT  33.580412  130.396361     NaN
1  east asia       japan   fukouka 1970-03-31  33.580412  130.396361     NaN
2  east asia      taiwan   unknown 1971-11-20  23.583333  119.583330     NaN
3  east asia       japan  yokosuka 1973-04-25  35.281341  139.672200     NaN
4  east asia       japan     tokyo 1973-08-29  35.689125  139.747742     NaN


In [62]:
# open file to get the shapefile of the countries
root = tk.Tk()
root.withdraw()

file_path = filedialog.askopenfilename(
    title="select your shapefile",
    filetypes=[("Shapefiles", "*.shp")]
)

# load the selected shapefile of the countries
gdf = gpd.read_file(file_path, encoding='latin1')

# use the existing df_attacks (assumed already loaded and cleaned)

# create a folium map and center it based on the shapefile
bounds = gdf.total_bounds  # [minx, miny, maxx, maxy]
center_lat = (bounds[1] + bounds[3]) / 2
center_lon = (bounds[0] + bounds[2]) / 2

# make the map
m = folium.Map(location=[center_lat, center_lon], zoom_start=3)

# add shapefile features
for idx, row in gdf.iterrows():
    geom = row['geometry']
    
    if isinstance(geom, Point):
        folium.CircleMarker(
            location=[geom.y, geom.x],
            radius=3,
            color='blue',
            fill=True,
            fill_opacity=0.6,
            popup=f"point - {row.get('city', 'unknown')}"
        ).add_to(m)
        
    elif isinstance(geom, Polygon):
        folium.Polygon(
            locations=[(coord[1], coord[0]) for coord in geom.exterior.coords],
            color='green',
            fill=True,
            fill_opacity=0.4,
            popup="polygon area"
        ).add_to(m)
        
    elif isinstance(geom, MultiPolygon):
        for poly in geom:
            folium.Polygon(
                locations=[(coord[1], coord[0]) for coord in poly.exterior.coords],
                color='purple',
                fill=True,
                fill_opacity=0.4,
                popup="multipolygon area"
            ).add_to(m)

    elif isinstance(geom, LineString):
        folium.PolyLine(
            locations=[(coord[1], coord[0]) for coord in geom.coords],
            color='red',
            weight=2,
            popup="line path"
        ).add_to(m)

# add attack locations on the same map
for idx, row in df.iterrows():
    folium.CircleMarker(
        location=[row['latitude'], row['longitude']],
        radius=2,
        color='red',
        fill=True,
        fill_opacity=0.7,
        popup=folium.Popup(
            f"city: {row.get('city', 'unknown')}<br>"
            f"country: {row.get('country', 'unknown')}<br>"
            f"date: {row.get('date', 'unknown')}",
            max_width=300
        )
    ).add_to(m)

# save the combined map
output_map = "generated_map.html"
m.save(output_map)

# open the html map automatically
webbrowser.open(output_map)

True

In [63]:
# Check CRS
print(gdf.crs)

# If not EPSG:4326, reproject it:
if gdf.crs != 'EPSG:4326':
    gdf = gdf.to_crs(epsg=4326)


EPSG:4326


In [64]:
#start of sattelite imagery intergration use the already loaded shape file
# open file 
root = tk.Tk()
root.withdraw()

# let user select a shapefile
shapefile = filedialog.askopenfilename(title="Pick a shapefile", filetypes=[("Shapefile", "*.shp")])

# read the shapefile
if shapefile:
    countries = gpd.read_file(shapefile)
    countries = countries.to_crs(epsg=4326)
    print("File loaded.")
else:
    print("No file selected.")

try:
    print("Using existing shapefile.")
    print(f"Number of features: {len(countries)}")
except NameError:
    print("Error: 'countries' is not defined. Please load the shapefile first.")



File loaded.
Using existing shapefile.
Number of features: 13


In [65]:
print(countries.columns)
print(countries.head(11))


Index(['id', 'Name', 'descriptio', 'timestamp', 'begin', 'end', 'altitudeMo',
       'tessellate', 'extrude', 'visibility', 'drawOrder', 'icon', 'geometry'],
      dtype='object')
    id         Name  descriptio  timestamp  begin  end  altitudeMo  \
0  NaN      Vietnam         NaN        NaN    NaN  NaN         NaN   
1  NaN       Taiwan         NaN        NaN    NaN  NaN         NaN   
2  NaN  Phillipines         NaN        NaN    NaN  NaN         NaN   
3  NaN     Malaysia         NaN        NaN    NaN  NaN         NaN   
4  NaN     Thailand         NaN        NaN    NaN  NaN         NaN   
5  NaN     Cambodia         NaN        NaN    NaN  NaN         NaN   
6  NaN         Laos         NaN        NaN    NaN  NaN         NaN   
7  NaN        Japan         NaN        NaN    NaN  NaN         NaN   
8  NaN  North Korea         NaN        NaN    NaN  NaN         NaN   
9  NaN        China         NaN        NaN    NaN  NaN         NaN   
10 NaN    Indonesia         NaN        NaN    NaN 

In [66]:
# show all unique names in the Name column
print(countries['Name'].unique())
print(countries.head(5))


['Vietnam' 'Taiwan' 'Phillipines' 'Malaysia' 'Thailand' 'Cambodia' 'Laos'
 'Japan' 'North Korea' 'China' 'Indonesia' 'Macau' 'Hong Kong']
   id         Name  descriptio  timestamp  begin  end  altitudeMo  tessellate  \
0 NaN      Vietnam         NaN        NaN    NaN  NaN         NaN           1   
1 NaN       Taiwan         NaN        NaN    NaN  NaN         NaN           1   
2 NaN  Phillipines         NaN        NaN    NaN  NaN         NaN           1   
3 NaN     Malaysia         NaN        NaN    NaN  NaN         NaN           1   
4 NaN     Thailand         NaN        NaN    NaN  NaN         NaN           1   

   extrude  visibility  drawOrder  icon  \
0        0          -1        NaN   NaN   
1        0          -1        NaN   NaN   
2        0          -1        NaN   NaN   
3        0          -1        NaN   NaN   
4        0          -1        NaN   NaN   

                                            geometry  
0  POLYGON Z ((104.7234 8.40533 0, 104.48061 10.3...  
1  POL

In [67]:
# get the bounds for a  polygon in the shapefile


# set the name of the feature to extract
feature_name = "hong kong"  

# filter the countries data using the name column
outline = countries[countries['Name'].str.lower() == feature_name]

# check if the feature exists
if outline.empty:
    print("no feature found with that name")
else:
    # get the bounds [minx, miny, maxx, maxy]
    bounds = outline.total_bounds
    print("bounds for", feature_name)
    print("minx:", bounds[0])
    print("miny:", bounds[1])
    print("maxx:", bounds[2])
    print("maxy:", bounds[3])
    
    # create the map centered on the feature
    center_lat = (bounds[1] + bounds[3]) / 2
    center_lon = (bounds[0] + bounds[2]) / 2
    m = folium.Map(location=[center_lat, center_lon], zoom_start=6)

    # add a rectangle for the bounding box
    folium.Rectangle(
        bounds=[[bounds[1], bounds[0]], [bounds[3], bounds[2]]],  # [[southwest], [northeast]]
        color='orange',
        fill=False,
        weight=3,
        dash_array='5, 5',
        tooltip=f"bounding box for {feature_name}"
    ).add_to(m)

    # add the actual polygon outline too
    folium.GeoJson(outline).add_to(m)

    # show the map
    m.save("bounding_box_map.html")
    #import webbrowser
    webbrowser.open("bounding_box_map.html")


bounds for hong kong
minx: 113.818832485244
miny: 22.10321497315797
maxx: 114.4280142022764
maxy: 22.5719524150113


In [68]:
# gets the minimum rotated rectangle that covers the outline from course code
search_area = outline.minimum_rotated_rectangle

search_area # again, in a jupyter notebook, this displays the polygon

<bound method GeoPandasBase.minimum_rotated_rectangle of     id       Name  descriptio  timestamp  begin  end  altitudeMo  tessellate  \
12 NaN  Hong Kong         NaN        NaN    NaN  NaN         NaN           1   

    extrude  visibility  drawOrder  icon  \
12        0          -1        NaN   NaN   

                                             geometry  
12  POLYGON Z ((114.24435 22.57195 0, 114.34411 22...  >

In [69]:
earthaccess.login(strategy='.netrc')

In [70]:
help(earthaccess.search_data)

Help on function search_data in module earthaccess.api:

search_data(count: int = -1, **kwargs: Any) -> List[earthaccess.results.DataGranule]
    Search dataset granules using NASA's CMR.

    [https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html](https://cmr.earthdata.nasa.gov/search/site/docs/search/api.html)

    Parameters:
        count: Number of records to get, -1 = all
        kwargs (Dict):
            arguments to CMR:

            * **short_name**: dataset short name, e.g. ATL08
            * **version**: dataset version
            * **doi**: DOI for a dataset
            * **daac**: e.g. NSIDC or PODAAC
            * **provider**: particular to each DAAC, e.g. POCLOUD, LPDAAC etc.
            * **temporal**: a tuple representing temporal bounds in the form
              `(date_from, date_to)`
            * **bounding_box**: a tuple representing spatial bounds in the form
              `(lower_left_lon, lower_left_lat, upper_right_lon, upper_right_lat)`

    Retu

In [71]:
# get the minimum rotated rectangle that covers the outline
search_area = outline.minimum_rotated_rectangle()

# extract the polygon from the GeoSeries
search_area = search_area.iloc[0]

# force the polygon into counter-clockwise orientation (required by CMR)
search_area = orient(search_area, sign=1.0)

# get the corner coordinates
coords = list(search_area.exterior.coords)

# search for datasets using these CCW coords
datasets = earthaccess.search_datasets(
    keyword='elevation',
    polygon=coords
)


In [79]:
print("Searching for MODIS Terra data within selected county...")
results = earthaccess.search_data(
    short_name="MOD13Q1",
    cloud_hosted=True,
    bounding_box=minimum_rotated_rectangle,
    temporal=("2000-01-01", "2023-01-31")
)

Searching for MODIS Terra data within selected county...


NameError: name 'minimum_rotated_rectangle' is not defined